<a href="https://colab.research.google.com/github/Attent1/CP2-IA/blob/main/CP2-IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DICIONARIO**


totalAtivo - O ativo total de uma empresa é a soma de todos os seus ativos, ou seja, seus bens e direitos. Dessa forma, inclui o ativo circulante, realizável a longo prazo e permanente, também chamado de não circulante. A definição de ativo na contabilidade é o conjunto de bens e direitos da organização.



In [241]:
# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

# Bibliotecas para uso e visualização de dados
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import para divisão dos dados de treino e teste
from sklearn.model_selection import train_test_split

# Imports para preparação de dados
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


# Import de métricas para avaliação dos modelos
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report
from sklearn.metrics import confusion_matrix


In [242]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [243]:
url = "/content/drive/MyDrive/solicitacoescredito.csv"
df = pd.read_csv(url)
df_sub = df[["margemBruta",  "faturamentoBruto", "totalAtivo"]]
df_sub.shape

(8973, 3)

In [244]:
esta_nula = df_sub['margemBruta'].isnull()
print(len(esta_nula))
esta_nula.head(8973)

8973


0       False
1        True
2       False
3       False
4       False
        ...  
8968     True
8969     True
8970     True
8971     True
8972     True
Name: margemBruta, Length: 8973, dtype: bool

In [245]:
df_sub.head()

,margemBruta,faturamentoBruto,totalAtivo
0,0.0,1766880.0,NaN
1,NaN,NaN,NaN
2,0.0,2814940.0,NaN
3,0.0,1285274.0,NaN
4,81459809.0,40779757.0,25793410.0


In [246]:
# df = df_sub.interpolate()
# Preencher os valores nulos com a média das colunas
df_sub['margemBruta'].fillna(df_sub['margemBruta'].mean(), inplace=True)
df_sub['totalAtivo'].fillna(df_sub['totalAtivo'].mean(), inplace=True)
df_sub['faturamentoBruto'].fillna(df_sub['faturamentoBruto'].mean(), inplace=True)


In [247]:
df.shape

(8973, 38)

In [248]:
#Filtrando registros com valores negativos nas colunas ativoCirculante e margemBruta
linhas_totalAtivo_margemBruta_faturamentoBruto = df_sub[(df_sub['totalAtivo'] >= 0) & (df_sub['margemBruta'] >= 0) & (df_sub['faturamentoBruto'] >= 0)]
print("Colunas onde 'totalAtivo e margemBruta' é maior ou igual a zero:")
print(linhas_totalAtivo_margemBruta_faturamentoBruto)


Colunas onde 'totalAtivo e margemBruta' é maior ou igual a zero:
       margemBruta  faturamentoBruto    totalAtivo
0     0.000000e+00      1.766880e+06  7.073623e+07
1     1.620988e+07      5.597420e+07  7.073623e+07
2     0.000000e+00      2.814940e+06  7.073623e+07
3     0.000000e+00      1.285274e+06  7.073623e+07
4     8.145981e+07      4.077976e+07  2.579341e+07
...            ...               ...           ...
8968  1.620988e+07      5.597420e+07  7.073623e+07
8969  1.620988e+07      5.597420e+07  7.073623e+07
8970  1.620988e+07      5.597420e+07  7.073623e+07
8971  1.620988e+07      5.597420e+07  7.073623e+07
8972  1.620988e+07      5.597420e+07  7.073623e+07

[8904 rows x 3 columns]


In [249]:
linhas_totalAtivo_margemBruta_faturamentoBruto.shape

(8904, 3)

In [250]:
linhas_totalAtivo_margemBruta_faturamentoBruto.describe()

,margemBruta,faturamentoBruto,totalAtivo
count,8.904000e+03,8.904000e+03,8.904000e+03
mean,1.651778e+07,5.627247e+07,7.114917e+07
std,1.113560e+08,3.213487e+08,6.472885e+08
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,1.359992e+06,4.013971e+06
50%,1.368395e+05,4.413127e+06,7.073623e+07
75%,7.030115e+06,2.710257e+07,7.073623e+07
max,3.366843e+09,6.426115e+09,5.482350e+10


In [251]:
#Adicionando condições para classificar o porte de cada empresa
condicoes = [
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 203377.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 1068988.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 2278600.0 ),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 2332812.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 4616340.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 80232850.0 ),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 10139079.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 1928754.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 31613710.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 3366842514.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 548235000.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 6426115000.0),


    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 203377.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 2332812.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 10139079.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 3366842514.0)
]

opcoes = ['muito pequena', 'pequena', 'grande', 'muito grande', 'muito pequena', 'pequena', 'grande', 'muito grande']

linhas_totalAtivo_margemBruta_faturamentoBruto['porte'] = pd.Series(np.select(condicoes, opcoes))

In [252]:
condicoes_grupo_1 = [
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 203377.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 1068988.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 2278600.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 2332812.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 4616340.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 80232850.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 10139079.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 1928754.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 31613710.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 3366842514.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['totalAtivo'] <= 548235000.0) & (linhas_totalAtivo_margemBruta_faturamentoBruto['faturamentoBruto'] <= 6426115000.0)
]

condicoes_grupo_2 = [
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 203377.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 2332812.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 10139079.0),
    (linhas_totalAtivo_margemBruta_faturamentoBruto['margemBruta'] <= 3366842514.0)
]

opcoes_grupo_1 = ['muito pequena', 'pequena', 'grande', 'muito grande']
opcoes_grupo_2 = ['muito pequena', 'pequena', 'grande', 'muito grande']

grupo_1 = np.select(condicoes_grupo_1, opcoes_grupo_1)
grupo_2 = np.select(condicoes_grupo_2, opcoes_grupo_2)

grupo_final = np.where(grupo_1 != None, grupo_1, grupo_2)

linhas_totalAtivo_margemBruta_faturamentoBruto['porte'] = grupo_final

In [253]:
#DataSet com a coluna grupo adicionada e os devidos valores
linhas_totalAtivo_margemBruta_faturamentoBruto.head(8900)

,margemBruta,faturamentoBruto,totalAtivo,porte
0,0.000000e+00,1.766880e+06,7.073623e+07,muito grande
1,1.620988e+07,5.597420e+07,7.073623e+07,muito grande
2,0.000000e+00,2.814940e+06,7.073623e+07,muito grande
3,0.000000e+00,1.285274e+06,7.073623e+07,muito grande
4,8.145981e+07,4.077976e+07,2.579341e+07,muito grande
...,...,...,...,...
8964,1.620988e+07,5.597420e+07,7.073623e+07,muito grande
8965,1.620988e+07,5.597420e+07,7.073623e+07,muito grande
8966,1.620988e+07,5.597420e+07,7.073623e+07,muito grande
8967,1.620988e+07,5.597420e+07,7.073623e+07,muito grande


In [254]:
esta_nula = linhas_totalAtivo_margemBruta_faturamentoBruto['porte'].isnull()
print(len(esta_nula))

8904


In [255]:
linhas_totalAtivo_margemBruta_faturamentoBruto.isnull().sum()

margemBruta         0
faturamentoBruto    0
totalAtivo          0
porte               0
dtype: int64

In [256]:
linhas_totalAtivo_margemBruta_faturamentoBruto.shape

(8904, 4)

In [257]:
linhas_totalAtivo_margemBruta_faturamentoBruto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8904 entries, 0 to 8972
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   margemBruta       8904 non-null   float64
 1   faturamentoBruto  8904 non-null   float64
 2   totalAtivo        8904 non-null   float64
 3   porte             8904 non-null   object 
dtypes: float64(3), object(1)
memory usage: 347.8+ KB


In [ ]:
#correlation_matrix = df_filtrado.corr()
correlation_matrix = df_sub.corr()

In [ ]:
plt.figure(figsize=(15, 12))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlação")
plt.show()

In [259]:
df = linhas_totalAtivo_margemBruta_faturamentoBruto.dropna()

In [260]:
df.shape

(8904, 4)

In [261]:
df.isnull().sum()

margemBruta         0
faturamentoBruto    0
totalAtivo          0
porte               0
dtype: int64

In [262]:
X = df.drop(columns=["porte"])
y = df["porte"]

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

# Padronização dos dados
ss = StandardScaler()
X_train_scaled = ss.fit_transform(X_train)
X_test_scaled = ss.transform(X_test)

In [264]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)

In [265]:
rf.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

In [266]:
y_pred = rf.predict(X_test)

In [267]:
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo Random Forest: {:.2f}".format(accuracy))

Acurácia do modelo Random Forest: 1.00


In [268]:
gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

In [269]:
gbm.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [270]:
y_pred = gbm.predict(X_test)

In [271]:
accuracy = accuracy_score(y_test, y_pred)
print("Acurácia do modelo Gradient Boosting: {:.2f}".format(accuracy))

Acurácia do modelo Gradient Boosting: 1.00


In [272]:
lda = LinearDiscriminantAnalysis()

lda.fit(X_train, y_train)

LinearDiscriminantAnalysis()

In [273]:
y_predicoes = lda.predict(X_test)

In [274]:
accuracy_score(y_true = y_test, y_pred = y_predicoes)

0.7739520958083832

In [285]:
input_values = [[203377.0, 1068988.0, 2278600.0], [3366842514.0, 548235000.0, 6426115000.0], [10139079.0, 1928754.0, 31613710.0]]  # Substitua pelos seus próprios valores

# Testando o modelo com os valores de entrada
for values in input_values:
    prediction = test_model(rf, values)
    print("Valores de entrada:", values)
    print("Previsão de porte:", prediction)
    print()

Valores de entrada: [203377.0, 1068988.0, 2278600.0]
Previsão de porte: ['pequena']

Valores de entrada: [3366842514.0, 548235000.0, 6426115000.0]
Previsão de porte: ['0']

Valores de entrada: [10139079.0, 1928754.0, 31613710.0]
Previsão de porte: ['muito grande']

